# Import

In [ ]:
import pandas as pd

# URL ของ Google Sheet
sheet_url = ""

# แก้ไข URL เพื่อให้ export เป็นไฟล์ CSV
csv_export_url = sheet_url.replace("/edit?usp=sharing", "/export?format=csv")

# 3. ใช้ pandas อ่านไฟล์ CSV จาก URL ที่แก้ไขแล้ว
try:
    df = pd.read_csv(csv_export_url)
    print("ดึงข้อมูลสำเร็จ")
    # แสดง 5 แถวแรกของข้อมูล
    print(df.head())

except Exception as e:
    print(f"เกิดข้อผิดพลาด: {e}")
    print("โปรดตรวจสอบว่า URL ถูกต้อง และชีตได้ถูกแชร์เป็นสาธารณะแล้ว")

In [ ]:
df.shape

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
# ดูจำนวน row ที่มี NaN ในแต่ละคอลัมน์
print(df.isnull().sum())

# ดูจำนวน row ทั้งหมดใน DataFrame ที่มีอย่างน้อยหนึ่งค่า NaN
print("Total rows with NaN:", df.isnull().any(axis=1).sum())

# Preprocess

In [ ]:
# Drop rows with missing values
df_cleaned = df.dropna()
df = df_cleaned.copy()

In [ ]:
df.info()

In [ ]:
df

In [ ]:
# Let pandas infer the format for each date
df['new_datetime'] = pd.to_datetime(df['date'], format='mixed')

In [ ]:
# Drop unwanted columns
df = df.drop(columns=['local_event', 'other_event']) #'local_event', 'other_event' เป็นคอลัมน์ที่ถูกเพิ่มมาทีหลังได้ไม่นาน อาจจะทำให้ประสิทธิภาพการวิจัยลดลง

In [ ]:
# Remove commas from numeric strings
for col in df.columns:
   if df[col].dtype == 'object':
         df[col] = df[col].astype(str).str.replace(',', '', regex=False)

In [ ]:
# Convert columns to appropriate data types
df['day'] = df['day'].astype('object')
df['date'] = pd.to_datetime(df['date'], format='mixed')
df['cost_change'] = pd.to_numeric(df['cost_change'], errors='coerce').astype('float64')
df['cost_material'] = pd.to_numeric(df['cost_material'], errors='coerce').astype('float64')
df['cost_cash'] = pd.to_numeric(df['cost_cash'], errors='coerce').astype('float64')
df['cost_banking'] = pd.to_numeric(df['cost_banking'], errors='coerce').astype('float64')
df['cost_sum'] = pd.to_numeric(df['cost_sum'], errors='coerce').astype('float64')
df['sales_fs'] = pd.to_numeric(df['sales_fs'], errors='coerce').astype('float64')
df['sales_qr'] = pd.to_numeric(df['sales_qr'], errors='coerce').astype('float64')
df['sales_gb'] = pd.to_numeric(df['sales_gb'], errors='coerce').astype('float64')
df['sales_sp'] = pd.to_numeric(df['sales_sp'], errors='coerce').astype('float64')
df['sales_lm'] = pd.to_numeric(df['sales_lm'], errors='coerce').astype('float64')
df['sales_rb'] = pd.to_numeric(df['sales_rb'], errors='coerce').astype('float64')
df['sales_sum'] = pd.to_numeric(df['sales_sum'], errors='coerce').astype('float64')
df['profit'] = pd.to_numeric(df['profit'], errors='coerce').astype('float64')
df['promotion_01'] = df['promotion_01'].astype('object')
df['promotion_02'] = df['promotion_02'].astype('object')
df['ismarketday'] = pd.to_numeric(df['ismarketday'], errors='coerce').fillna(0).astype('int64')
df['isschoolday'] = pd.to_numeric(df['isschoolday'], errors='coerce').fillna(0).astype('int64')
df['holiday'] = pd.to_numeric(df['holiday'], errors='coerce').fillna(0).astype('int64')
df['isbuddaday'] = pd.to_numeric(df['isbuddaday'], errors='coerce').fillna(0).astype('int64')
df['nd_lottery'] = pd.to_numeric(df['nd_lottery'], errors='coerce').fillna(0).astype('int64')
df['applylkcolor'] = df['applylkcolor'].astype('object')
df['PP'] = pd.to_numeric(df['PP'], errors='coerce').astype('float64')
df['T'] = pd.to_numeric(df['T'], errors='coerce').astype('float64')
df['H'] = pd.to_numeric(df['H'], errors='coerce').astype('float64')
df['V'] = pd.to_numeric(df['V'], errors='coerce').astype('float64')
df['pm2.5'] = pd.to_numeric(df['pm2.5'], errors='coerce').astype('float64')
df['pm10'] = pd.to_numeric(df['pm10'], errors='coerce').astype('float64')
df['o3'] = pd.to_numeric(df['o3'], errors='coerce').astype('float64')
df['no2'] = pd.to_numeric(df['no2'], errors='coerce').astype('float64')
df['pos_cash'] = pd.to_numeric(df['pos_cash'], errors='coerce').astype('float64')
df['pos_qr'] = pd.to_numeric(df['pos_qr'], errors='coerce').astype('float64')

# EDA

In [ ]:
# Merge banking cost into material cost
df['cost_material'] = df['cost_material'] + df['cost_banking']
df = df.drop(columns=['cost_banking'])

In [ ]:
# Merge QR sales into FS sales
df['sales_qr'] = df['sales_qr'].fillna(0)
df['sales_fs'] = df['sales_fs'].fillna(0)
df['sales_fs'] = df['sales_qr'] + df['sales_fs']
df = df.drop(columns=['sales_qr'])

In [ ]:
# Map Thai day abbreviations to English
day_mapping = {
            'อา.': 'Sun',
            'จ.': 'Mon',
            'อ.': 'Tue',
            'พ.': 'Wed',
            'พฤ.': 'Thu',
            'ศ.': 'Fri',
            'ส.': 'Sat'
        }
df['day'] = df['day'].replace(day_mapping)

In [ ]:
df = df[df['sales_sum'] != 0]

In [ ]:
# Adjust Friday sales by adding 6000
df.loc[df['day'] == 'Fri', 'sales_sum'] = df.loc[df['day'] == 'Fri', 'sales_sum'] + 5000

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df['date'], df['sales_sum'])
plt.xlabel('date')
plt.ylabel('sales_sum')
plt.title('Sales Sum over Time')
plt.grid(True)
plt.show()

In [ ]:
# Calculate the average sales_sum for each day of the week
average_sales_by_day = df.groupby('day')['sales_sum'].mean().reindex(['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat'])

# Plot the linear graph
plt.figure(figsize=(10, 6))
plt.plot(average_sales_by_day.index, average_sales_by_day.values, marker='o', linestyle='-')
plt.xlabel('Day of the Week')
plt.ylabel('Average Sales Sum')
plt.ylim(0)
plt.title('Average Sales Sum by Day of the Week')
plt.grid(True)
plt.show()

In [ ]:
# Select only the numeric columns
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
# Create a single figure and axes for the box plots
plt.figure(figsize=(15, 10))
# Plot box plots for each numeric column
df[numeric_cols].boxplot(vert=False, patch_artist=True, showfliers=True) # vert=False for horizontal box plots
plt.title('Box Plot of Numeric Variables')
plt.xlabel('Value')
plt.ylabel('Numeric Variables')
plt.tight_layout() # Adjust layout to prevent labels overlapping
plt.show()

In [ ]:
# Extract day of the week and year
df['dayofweek'] = df['new_datetime'].dt.dayofweek  # Monday=0, Sunday=6
df['year'] = df['new_datetime'].dt.year

# Filter out Saturday (dayofweek == 5)
df_filtered_days = df[df['dayofweek'] != 5].copy()

# Map dayofweek to English day names
day_map = {
    0: 'Mon',
    1: 'Tue',
    2: 'Wed',
    3: 'Thu',
    4: 'Fri',
    5: 'Sat',
    6: 'Sun'
}
df_filtered_days['day_name'] = df_filtered_days['dayofweek'].map(day_map)

# Get the list of days to plot (Sun to Fri)
days_to_plot = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri']

# Create subplots for each day
fig, axes = plt.subplots(nrows=len(days_to_plot), ncols=1, figsize=(12, 3 * len(days_to_plot)))
fig.suptitle('Seasonal Plot of Average Daily Sales Sum (Sun - Fri)', y=1.02)

for i, day in enumerate(days_to_plot):
    # Filter data for the current day
    df_day = df_filtered_days[df_filtered_days['day_name'] == day].copy()

    # Group by date and calculate the average sales sum for that date
    # (even though there's usually only one entry per date, this handles potential duplicates gracefully)
    daily_avg_sales = df_day.groupby('new_datetime')['sales_sum'].mean()

    # Plot the average daily sales for the current day
    axes[i].plot(daily_avg_sales.index, daily_avg_sales.values, label=day)
    axes[i].set_title(f'Average Sales Sum on {day}')
    axes[i].set_ylabel('Average Sales Sum')
    axes[i].grid(True)
    axes[i].legend()

# Set a common xlabel for all subplots
fig.text(0.5, 0.00, 'Date', ha='center')

plt.tight_layout()
plt.show()

In [ ]:
# Set the 'date' column as the index
df_decomp = df.set_index('date')

# Select the time series data for decomposition
time_series = df_decomp['sales_sum']

# Perform additive decomposition
decomposition = seasonal_decompose(time_series, model='additive', period=7) # Use period=7 for weekly seasonality

# Plot the decomposed components
fig = decomposition.plot()
fig.set_size_inches(10, 8)
plt.tight_layout()
plt.show()


In [ ]:
# เพื่อแยกตามเดือนตามปฏิทินจริงๆ เราสามารถ Resample ข้อมูลเป็นรายเดือนก่อน แล้วจึงทำ Decomposition
# Resample to monthly frequency, summing the sales_sum for each month
monthly_data = time_series.resample('M').sum()

# A better approach for visualizing calendar month patterns is a seasonal plot by month.

# We already have a 'new_datetime' column. Let's use that to extract the month.
df['month'] = df['new_datetime'].dt.month

# Group by month and calculate the average sales sum for each month across all years
average_sales_by_month = df.groupby('month')['sales_sum'].mean()

# Map month numbers to names for better plotting
month_names = {
    1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
    7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}
average_sales_by_month.index = average_sales_by_month.index.map(month_names)

# Plot the average sales by calendar month
plt.figure(figsize=(10, 6))
plt.plot(average_sales_by_month.index, average_sales_by_month.values, marker='o', linestyle='-')
plt.xlabel('Calendar Month')
plt.ylabel('Average Sales Sum')
plt.title('Average Sales Sum by Calendar Month Across All Years')
plt.grid(True)
plt.show()

# Let's do that as well, as it aligns more with the concept of decomposition plots.
# Resample to monthly frequency (sum of sales for each month)
monthly_series = time_series.resample('M').sum()

# Perform additive decomposition on the monthly aggregated data with period=12
# This will show trend and annual seasonality (12-month cycle) within the monthly sums
decomposition_monthly_agg = seasonal_decompose(monthly_series, model='additive', period=12)

# Plot the decomposed components for monthly aggregated data
fig_monthly_agg = decomposition_monthly_agg.plot()
fig_monthly_agg.set_size_inches(10, 8)
fig_monthly_agg.suptitle('Decomposition of Monthly Sales Sum (Annual Seasonality)', y=1.02) # Add a specific title
plt.tight_layout()
plt.show()

In [ ]:
# Group by year and month, then sum sales_sum
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

# Create monthly data grouped by year
monthly_sales_by_year = df.groupby(['year', 'month'])['sales_sum'].sum().reset_index()

# Get the maximum year in the dataset
max_year = monthly_sales_by_year['year'].max()

# Filter out the last month of the last year
monthly_sales_filtered = monthly_sales_by_year[
    ~((monthly_sales_by_year['year'] == max_year) & (monthly_sales_by_year['month'] == 12))
].copy()


plt.figure(figsize=(12, 7))

# Iterate through each year and plot a line
for year in monthly_sales_filtered['year'].unique():
    df_year = monthly_sales_filtered[monthly_sales_filtered['year'] == year]

    # Ensure all months (1-12) are present for consistent plotting
    # Create a full set of months for the year
    full_months = pd.DataFrame({'month': range(1, 13)})
    df_year_full = pd.merge(full_months, df_year, on='month', how='left')

    # Sort by month
    df_year_full = df_year_full.sort_values('month')

    # Plot the data, excluding the last month if it's the max year and month 12
    months_to_plot = range(1, 13)
    if year == max_year:
        months_to_plot = range(1, 12) # Exclude month 12 for the last year

    plt.plot(df_year_full['month'].iloc[months_to_plot.start-1 : months_to_plot.stop-1], # Adjust indexing for months
             df_year_full['sales_sum'].iloc[months_to_plot.start-1 : months_to_plot.stop-1],
             marker='o', linestyle='-', label=str(year))


# Set x-axis labels to month names
month_names_short = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
plt.xticks(range(1, 13), month_names_short) # Set ticks for all 12 months

plt.xlabel('Month')
plt.ylabel('Sum of Sales')
plt.ylim(ymin=0)
plt.title('Monthly Sales Sum by Year')
plt.legend(title='Year')
plt.grid(True)
plt.show()


In [ ]:
import seaborn as sns

# Select only the numeric columns again for correlation calculation
numeric_cols_for_corr = df.select_dtypes(include=['float64', 'int64']).columns

# Calculate the correlation matrix
correlation_matrix = df[numeric_cols_for_corr].corr()

# Print the correlation matrix
print("Correlation Matrix of Numeric Features:")
print(correlation_matrix)

# Optional: Visualize the correlation matrix using a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".1f", linewidths=.5)
plt.title('Correlation Heatmap of Numeric Features')
plt.show()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Select numerical features for VIF calculation
# Exclude the target variable ('sales_sum') and potentially highly correlated/dependent variables like profit, costs, etc.
# Also exclude variables that are essentially identifiers or not meant to be predictors like date, year, month, dayofweek, day_name
# Let's select relevant numerical predictors identified from EDA/correlation analysis.
# Based on the previous analysis, potential numerical predictors could include:
# ismarketday, isschoolday, holiday, isbuddaday, nd_lottery,
# sales from other channels (sales_gb, sales_rb) - but remember to use lagged values for prediction!
# Weather/pollution variables (T, H, V, PP, pm2.5, o3, no2) - consider multicollinearity

# For VIF calculation, we need the predictor variables.
# Let's start with a subset of potential numerical predictors excluding the target and cost/profit.
# Also, let's be mindful of multicollinearity issues identified (e.g., pm2.5 and pm10).
# We'll use a subset of potentially useful features for this example.
# Note: VIF is calculated for the independent variables *before* modeling.

# First, identify numerical columns that are potential predictors
# Exclude 'sales_sum', 'profit', and cost-related columns as they are results/components of sales.
# Exclude date-related columns that are not numerical predictors themselves but used for time series analysis.
# Exclude 'dayofweek', 'year', 'month' as we might use categorical encoding for time-based features or rely on the time series model itself.
numerical_predictors = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Remove the target variable and other dependent/related variables
exclude_cols = ['sales_sum', 'profit', 'cost_sum', 'cost_material', 'cost_change',
                'date', 'new_datetime', 'year', 'month', 'dayofweek']

vif_features = [col for col in numerical_predictors if col not in exclude_cols]

# Optional: Further refine vif_features based on EDA/correlation.
# For instance, if pm10 is highly correlated with pm2.5 and we decide to use only pm2.5.
# Let's remove pm10 for this VIF calculation based on previous observation (corr=0.86 with pm2.5).
if 'pm10' in vif_features:
    vif_features.remove('pm10')

# Create a DataFrame with only the features for VIF calculation
X = df[vif_features]

# Ensure there are no missing values in the selected columns for VIF calculation
X = X.dropna()

# Handle potential infinite values that might arise from perfectly correlated variables
# A common issue is when one variable is a linear combination of others.
# Check for constant columns or columns that are linear combinations.
# A simple check is to see if the variance is zero or close to zero.
non_constant_cols = X.columns[X.var() > 1e-6] # Remove columns with near-zero variance
X = X[non_constant_cols]

# If after removing near-zero variance columns, there are still perfect linear combinations,
# VIF calculation will fail. This might require domain knowledge or further analysis
# to identify and remove one of the perfectly correlated variables.
# For example, if 'A', 'B', 'C' are such that A + B = C, then including all three will cause issues.

# Calculate VIF for each predictor
# Make sure X is not empty
if not X.empty:
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    # calculating VIF for each feature
    vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                       for i in range(len(X.columns))]

    print("\nVariance Inflation Factor (VIF):")
    print(vif_data)
else:
    print("No suitable numerical features found for VIF calculation after exclusions.")



# Linear Regression

## Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle

## Preprocess

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
df.head()

In [ ]:
df_lr = df.copy()

In [ ]:
df_lr['is_weekend'] = (df_lr.index.dayofweek == 6).astype(int)
df_lr.head()

In [ ]:
df_lr['is_Sun'] = (df_lr.index.dayofweek == 6).astype(int)
df_lr['is_Mon'] = (df_lr.index.dayofweek == 0).astype(int)
df_lr['is_Tue'] = (df_lr.index.dayofweek == 1).astype(int)
df_lr['is_Wed'] = (df_lr.index.dayofweek == 2).astype(int)
df_lr['is_Thu'] = (df_lr.index.dayofweek == 3).astype(int)
df_lr['is_Fri'] = (df_lr.index.dayofweek == 4).astype(int)
df.head(3)

In [ ]:
df_lr['is_Jan'] = (df_lr.index.month == 1).astype(int)
df_lr['is_Feb'] = (df_lr.index.month == 2).astype(int)
df_lr['is_Mar'] = (df_lr.index.month == 3).astype(int)
df_lr['is_Apr'] = (df_lr.index.month == 4).astype(int)
df_lr['is_May'] = (df_lr.index.month == 5).astype(int)
df_lr['is_Jun'] = (df_lr.index.month == 6).astype(int)
df_lr['is_Jul'] = (df_lr.index.month == 7).astype(int)
df_lr['is_Aug'] = (df_lr.index.month == 8).astype(int)
df_lr['is_Sep'] = (df_lr.index.month == 9).astype(int)
df_lr['is_Oct'] = (df_lr.index.month == 10).astype(int)
df_lr['is_Nov'] = (df_lr.index.month == 11).astype(int)
df_lr['is_Dec'] = (df_lr.index.month == 12).astype(int)
df_lr.head(3)

In [ ]:
df_lr.dropna(inplace=True)
df_lr.head()

In [ ]:
# คำนวณ time_index จาก index ของ DataFrame
df_lr['time_index'] = (df_lr.index - df_lr.index.min()).days

df_lr.head()

In [ ]:
df_lr['lag_1'] = df_lr['sales_sum'].shift(1)
df_lr['lag_2'] = df_lr['sales_sum'].shift(2)
df_lr['lag_3'] = df_lr['sales_sum'].shift(3)
df_lr['lag_4'] = df_lr['sales_sum'].shift(4)
df_lr['lag_5'] = df_lr['sales_sum'].shift(5)
df_lr['lag_6'] = df_lr['sales_sum'].shift(6)
df_lr['lag_7'] = df_lr['sales_sum'].shift(7)
df_lr = df_lr.reindex(columns=['sales_sum', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7'
                         , 'year', 'time_index'
                         #, 'is_Jan', 'is_Feb', 'is_Mar', 'is_Apr', 'is_May', 'is_Jun', 'is_Jul', 'is_Aug', 'is_Sep', 'is_Oct', 'is_Nov', 'is_Dec'
                         #, 'ismarketday', 'isschoolday', 'holiday'
                         ])

df_lr.head()

In [ ]:
features_recursive = [
        'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7',
        'year', 'time_index'
        #, 'is_Jan', 'is_Feb', 'is_Mar', 'is_Apr', 'is_May', 'is_Jun', 'is_Jul', 'is_Aug', 'is_Sep', 'is_Oct', 'is_Nov', 'is_Dec'
        #, 'ismarketday', 'isschoolday', 'holiday'
    ]

df_lr = df_lr[features_recursive]

In [ ]:
import pickle

In [ ]:
# Load the saved model
loaded_model = pickle.load(open('Linear_Regression_model.pkl', 'rb'))
print("Model loaded successfully.")

# Get the last 7 lags from the original dataframe for prediction
last_7_lags = df_lr[['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7']].tail(1).values.flatten()

# Prepare future dates and calendar features for the next 7 days
last_date = df_lr.index[-1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=7, freq='D')
future_df_predict = pd.DataFrame(index=future_dates)
future_df_predict['year'] = future_df_predict.index.year
future_df_predict['time_index'] = range(df_lr['time_index'].max() + 1, df_lr['time_index'].max() + 1 + 7)

# Make recursive predictions
predictions_recursive_loaded = []
current_lags_loaded = last_7_lags.copy()

print("Starting 7-step recursive forecast using loaded model...")
for i in range(7):
    calendar_features_loaded = future_df_predict.iloc[i][['year', 'time_index']]
    input_vector_loaded = np.concatenate([current_lags_loaded, calendar_features_loaded.values]).reshape(1, -1)

    prediction_loaded = loaded_model.predict(input_vector_loaded)[0]
    predictions_recursive_loaded.append(prediction_loaded)

    current_lags_loaded = np.roll(current_lags_loaded, -1)
    current_lags_loaded[-1] = prediction_loaded
    print(f"Step {i+1}/7 predicted: {prediction_loaded:.2f}")

print("\nRecursive forecasting completed using loaded model.")

# Display the predictions with dates
predictions_df = pd.DataFrame({'Date': future_dates, 'Predicted Sales': predictions_recursive_loaded})
predictions_df = predictions_df.set_index('Date')
print("\n7-Day Sales Predictions:")
display(predictions_df)

In [ ]:
# Ensure df_original is loaded (assuming it's available from previous cells)
# If not, you might need to load it again
# df_original = pd.read_csv("/content/drive/MyDrive/swuds/t5_thesis/98_progress/dataset/nd_sales_LinearRegression_base.csv")
# df_original['date'] = pd.to_datetime(df_original['date'])
# df_original = df_original.set_index('date')

# Calculate the average sales_sum for each day of the week for the last 3 months
last_date_original = df.index.max()
three_months_ago = last_date_original - pd.DateOffset(months=3)
recent_df = df[df.index >= three_months_ago].copy() # Use copy to avoid SettingWithCopyWarning

average_sales_last_3_months = recent_df.groupby('day')['sales_sum'].mean().reindex(['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat'])

# Ensure predictions_df is available (assuming it's available from previous cells)
# If not, you might need to regenerate predictions or load from a file
# predictions_df = ... # Load or generate your predictions_df here

# Plot the linear graph
plt.figure(figsize=(10, 6))
plt.plot(average_sales_last_3_months.index, average_sales_last_3_months.values, marker='o', linestyle='-', color='blue', label='Average Sales Last 3 Months')

# Add the predicted sales to the plot
# Extract day of the week from predicted dates
predicted_days_of_week = predictions_df.index.day_name()
plt.plot(predicted_days_of_week, predictions_df['Predicted Sales'].values, marker='x', linestyle='-', color='red', label='Predicted Sales (Next 7 Days)')

plt.xlabel('Day of the Week')
plt.ylabel('Sales Sum')
plt.ylim(0)
plt.title('Sales Sum by Day of the Week (Average Last 3 Months vs Predicted)')
plt.legend()
plt.grid(True)
plt.show()

# Prophet

## Import

In [ ]:
#!pip install prophet

In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt
from prophet.diagnostics import cross_validation, performance_metrics
import itertools

## Preprocess

In [ ]:
df_pp = df.copy()

In [ ]:
# รีเซ็ต index เพื่อให้ 'date' กลับมาเป็นคอลัมน์
df_pp = df_pp.reset_index()

# เปลี่ยนชื่อคอลัมน์สำหรับ Prophet
df_pp = df_pp.rename(columns={'date': 'ds', 'sales_sum': 'y'})

# แสดงผล 5 แถวแรกเพื่อตรวจสอบ
df_pp.head()

In [ ]:
# กำหนดลิสต์ของ Regressor ที่จะใช้
regressor_columns = ['ismarketday', 'isschoolday', 'holiday']

# เลือกคอลัมน์ที่จำเป็นทั้งหมด
df_prophet = df_pp[['ds', 'y'] + regressor_columns].copy()
print("Data prepared for Prophet with regressors.")

In [ ]:
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

In [ ]:
import pickle
import pandas as pd
from prophet import Prophet

# Load the model
with open('Prophet_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

print("Prophet model loaded successfully.")

# Create a future dataframe for the next 7 days
# Find the last date in your original dataframe (df_prophet)
last_date = df_prophet['ds'].max()

# Generate future dates starting from the day after the last date
future_dates = loaded_model.make_future_dataframe(periods=9, freq='D', include_history=False) # Generate a bit more to ensure 7 non-Saturdays

# Filter out Saturdays
future_no_saturday = future_dates[future_dates['ds'].dt.dayofweek != 5]

# Take the first 7 future dates that are not Saturdays
future_7_days = future_no_saturday.head(7)

# Merge with the regressors from the original data for these future dates
# If you have future regressor values, you should load or generate them here.
# For demonstration, we will use the ffill method as in previous steps, assuming
# the regressor values from the last known date are carried forward.
future_with_regressors = pd.merge(future_7_days, df_prophet[['ds'] + regressor_columns], on='ds', how='left')

# Fill any missing regressor values (e.g., for dates beyond the original data)
future_with_regressors.fillna(method='ffill', inplace=True)


print("\nFuture dates for forecasting (next 7 non-Saturdays):")
print(future_with_regressors)

# Make predictions
future_forecast = loaded_model.predict(future_with_regressors)

print("\nForecast for the next 7 days:")
print(future_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

In [ ]:
# 1. Calculate the day of the week for each date in df_prophet
df_prophet['dayofweek'] = df_prophet['ds'].dt.dayofweek  # Monday=0, Sunday=6

# 2. Calculate the 3-month rolling average for each day of the week
# We need to calculate this for each day of the week separately to get the average for each specific day
# Create a temporary DataFrame for calculating rolling average
df_temp = df_prophet.copy()
df_temp['rolling_avg_3m'] = df_temp.groupby('dayofweek')['y'].transform(lambda x: x.rolling(window=12, min_periods=1).mean()) # Assuming roughly 4 weeks/month * 3 months = 12 data points

# Change this line to use .last()
# This creates a DataFrame with the last entry for each day of the week
average_sales_last_3_months = df_temp.groupby('dayofweek').last()

# The rest of your code from this point on will work correctly
# For example, when you reset the index, 'dayofweek' will become a column
average_sales_last_3_months_plot = average_sales_last_3_months.reset_index()

# This line will now succeed
average_sales_last_3_months_plot['day'] = average_sales_last_3_months_plot['dayofweek'].map(day_map)

# Sort by day of the week
average_sales_last_3_months = average_sales_last_3_months.sort_index()

# 4. Prepare the forecast data for plotting (last 7 days forecast)
# We already have 'future_forecast' from the previous step which contains the next 7 days forecast
# Ensure the forecast data is also sorted by day of the week for consistent plotting
future_forecast['dayofweek'] = future_forecast['ds'].dt.dayofweek
forecast_for_plot = future_forecast.sort_values(by='dayofweek')

# Ensure we only have one forecast point per day of the week if there are duplicates
forecast_for_plot = forecast_for_plot.drop_duplicates(subset=['dayofweek'], keep='first')


# 5. Plotting
plt.figure(figsize=(10, 6))

# Define the order of days of the week for plotting
day_order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sun']
day_map = {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri', 6: 'Sun'}

# Plot Average Sales (last 3 months)
# Map the dayofweek index to the string representation
average_sales_last_3_months_plot = average_sales_last_3_months.reset_index()
average_sales_last_3_months_plot['day'] = average_sales_last_3_months_plot['dayofweek'].map(day_map)
average_sales_last_3_months_plot = average_sales_last_3_months_plot.set_index('day').loc[day_order]

# Ensure that all days in day_order are present in average_sales_last_3_months_plot before plotting
# If a day is missing, its value will be NaN, which matplotlib handles correctly for line plots
plt.plot(average_sales_last_3_months_plot.index, average_sales_last_3_months_plot['rolling_avg_3m'], marker='o', linestyle='-', color='blue', label='Average Sales (Last 3 Months)')

# Plot Prophet Forecast
forecast_for_plot['day'] = forecast_for_plot['dayofweek'].map(day_map)
forecast_for_plot = forecast_for_plot.set_index('day').loc[day_order]

# Ensure that all days in day_order are present in forecast_for_plot before plotting
plt.plot(forecast_for_plot.index, forecast_for_plot['yhat'], marker='x', linestyle='-', color='red', label='Prophet Forecast')

# Set labels and title
plt.xlabel('Day of the Week')
plt.ylabel('Sales')
plt.title('Average Sales (Last 3 Months) vs. Prophet Forecast by Day of the Week')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()